In [ ]:
import pandas as pd
import torch

In [ ]:
predictions = pd.read_csv("dataset/validations/predictions.csv")
predictions = predictions.rename(
    columns={"x_min": "xmin", "y_min": "ymin", "x_max": "xmax", "y_max": "ymax"}
)
predictions.sample(5)

In [ ]:
converted_labels = predictions["Image_ID"].unique()

In [ ]:
def convert_df(df: pd.DataFrame):
	df = df.copy().dropna()
	return {
		img_id: {
			"boxes": torch.tensor(raw[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float32),
			"scores": (
				torch.tensor(raw["confidence"].values, dtype=torch.float32)
				if "confidence" in raw.columns
				else None
			),
			"labels": torch.tensor(raw["class_id"].values, dtype=torch.int32),
		}
		for (img_id, ), raw in df.groupby(["Image_ID"])
	}

def default_value():
	return {
		"boxes": torch.empty((0, 4), dtype=torch.float32),
		"scores": torch.empty((0,), dtype=torch.float32),
		"labels": torch.empty((0,), dtype=torch.int32),
	}

def get_preds_data(preds, thr: float = 0.5):
	if thr is not None:
		preds = preds[preds["confidence"] >= thr]
	preds = convert_df(preds)
	d = default_value()
	return {i: preds.get(i, d) for i in converted_labels}

In [ ]:
preds = get_preds_data(predictions, thr=None)
len(preds)

In [ ]:
validations = pd.read_csv("dataset/Val_df.csv")
validations.sample(5)

In [ ]:
vals = get_preds_data(validations, thr=None)
len(vals)

In [ ]:
vals["ID_B9K2SI.jpg"]

In [ ]:
import torch

def calculate_iou_tensor(box1, box2):
	"""
	box1: [4], box2: [4]
	Format: [xmin, ymin, xmax, ymax]
	"""
	xA = torch.max(box1[0], box2[0])
	yA = torch.max(box1[1], box2[1])
	xB = torch.min(box1[2], box2[2])
	yB = torch.min(box1[3], box2[3])

	inter_area = torch.clamp(xB - xA, min=0) * torch.clamp(yB - yA, min=0)
	box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
	box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
	union_area = box1_area + box2_area - inter_area
	return inter_area / union_area if union_area > 0 else torch.tensor(0.0)

def evaluate_detection(predictions, ground_truths, iou_threshold=0.5, conf_threshold=0.0):
	"""
	predictions: list of dicts (len = batch size), each dict with 'boxes', 'scores', 'labels'
	ground_truths: list of dicts with 'boxes', 'labels'
	"""
	TP = 0
	FP = 0
	FN = 0

	for preds, gts in zip(predictions, ground_truths):
		pred_boxes = preds['boxes']
		pred_labels = preds['labels']
		pred_scores = preds['scores'] if preds['scores'] is not None else torch.ones(len(pred_boxes))

		gt_boxes = gts['boxes']
		gt_labels = gts['labels']
		matched_gt = set()

		for i in range(len(pred_boxes)):
			if pred_scores[i] < conf_threshold:
				continue
			pred_box = pred_boxes[i]
			pred_label = pred_labels[i]
			match_found = False

			for j in range(len(gt_boxes)):
				if j in matched_gt:
					continue
				if pred_label != gt_labels[j]:
					continue
				iou = calculate_iou_tensor(pred_box, gt_boxes[j])
				if iou >= iou_threshold:
					TP += 1
					matched_gt.add(j)
					match_found = True
					break
			if not match_found:
				FP += 1

		FN += len(gt_boxes) - len(matched_gt)

	precision = TP / (TP + FP) if (TP + FP) else 0.0
	recall = TP / (TP + FN) if (TP + FN) else 0.0
	f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0
	accuracy = TP / (TP + FP + FN) if (TP + FP + FN) else 0.0

	return {
		'TP': TP,
		'FP': FP,
		'FN': FN,
		'Precision': precision,
		'Recall': recall,
		'F1 Score': f1_score,
		'Accuracy': accuracy
	}

In [ ]:
import numpy as np

for i in np.linspace(0.0, 0.95, 15):
	scores = evaluate_detection(
		preds.values(),
		vals.values(),
		iou_threshold=0.5,
		conf_threshold=i
	)
	print("Evaluation metric at:", i, " score :", scores)

In [ ]:
import torch
from torchmetrics.detection import MeanAveragePrecision

def compute_map(preds, targets, iou_thresholds):
	"""
	Compute mAP at different IoU thresholds using torchmetrics.
	
	Args:
		preds: List of dicts with 'boxes', 'scores', 'labels' for predictions
		targets: List of dicts with 'boxes', 'labels' for ground truth
		iou_thresholds: List of IoU thresholds to evaluate
	
	Returns:
		Dict containing mAP results for each IoU threshold
	"""
	# Initialize the metric
	metric = MeanAveragePrecision(iou_thresholds=iou_thresholds)
	
	# Update metric with predictions and targets
	metric.update(preds, targets)
	
	# Compute the results
	result = metric.compute()
	
	return result

In [ ]:
thrs = np.linspace(0.0, 0.95, 15)
# Example usage
# if __name__ == "__main__":
# Example predictions and targets
targets = list(vals.values())

iou_thresholds = [0.5]
for i in thrs:
	preds = list(get_preds_data(predictions, i).values())

	# Compute mAP
	results = compute_map(preds, targets, iou_thresholds)

	# Print results
	print("mAP Results:", i, " - ", results)